In [1]:
#!pip install compiler_gym 'ray[default,rllib]' &>/dev/null || echo "Install failed!"

import compiler_gym
import ray

from ray.rllib.agents.ppo import PPOTrainer
from compiler_gym.wrappers import ConstrainedCommandline, TimeLimit
from ray import tune
from itertools import islice
from compiler_gym.wrappers import CycleOverBenchmarks
from compiler_gym.util.registration import register

import loop_tool_service

from service_py.datasets import loop_tool_dataset
from service_py.rewards import flops_loop_nest_reward, flops_reward, runtime_reward
import wandb
wandb.init(project="loop_tool", entity="dejang", sync_tensorboard=True)

/home/dejang/anaconda3/envs/compiler_gym/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dejang. Use `wandb login --relogin` to force relogin


In [2]:
def make_env() -> compiler_gym.envs.CompilerEnv:
    """Make the reinforcement learning environment for this experiment."""
    
    env = loop_tool_service.make(
        "loop_tool_env-v0",
        observation_space="5_prev_actions_tensor",
        reward_space="flops_loop_nest_tensor",
    )

    env = TimeLimit(env, max_episode_steps=10)
    return env

In [3]:
with make_env() as env:
    print("Action space:", env.action_space)
    print("Observation space:", env.observation_space)
    print("Reward space:", env.reward_space)

Action space: NamedDiscrete([up, down, swap_up, swap_down])
Observation space: Box([[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], [[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]], (1, 20), float32)
Reward space: flops_loop_nest_tensor


In [4]:
with make_env() as env:
    # The two datasets we will be using:
    lt_dataset = env.datasets["loop_tool_simple-v0"]
    # train_benchmarks = list(islice(lt_dataset.benchmarks(), 1))
    # test_benchmarks = list(islice(lt_dataset.benchmarks(), 2))
    
    bench = ["benchmark://loop_tool_simple-v0/simple"]
            #  "benchmark://loop_tool_simple-v0/mm128", 
            #  "benchmark://loop_tool_simple-v0/mm"] 

    train_benchmarks = bench
    test_benchmarks = bench

print("Number of benchmarks for training:", len(train_benchmarks))
print("Number of benchmarks for testing:", len(test_benchmarks))


Number of benchmarks for training: 1
Number of benchmarks for testing: 1


In [5]:
def make_training_env(*args) -> compiler_gym.envs.CompilerEnv:
    """Make a reinforcement learning environment that cycles over the
    set of training benchmarks in use.
    """
    del args  # Unused env_config argument passed by ray
    return CycleOverBenchmarks(make_env(), train_benchmarks)


In [6]:
env = make_training_env()
env.reset()

E0705 15:25:02.042448 140184553244224 example_service.py:263] CRITICAL - 

Working_dir = /dev/shm/compiler_gym_dejang/s/0705T152501-249505-dbfd



array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]], dtype=float32)

In [7]:
if ray.is_initialized():
    ray.shutdown()
ray.init(include_dashboard=False, ignore_reinit_error=True)

tune.register_env("compiler_gym", make_training_env)

In [8]:
import time
from ray import tune
from ray.tune import Stopper

class TimeStopper(Stopper):
    def __init__(self):
        self._start = time.time()
        self._deadline = 30

    def __call__(self, trial_id, result):
        return False

    def stop_all(self):
        return time.time() - self._start > self._deadline


In [9]:
PPO_CONFIG = {
    "log_level": "ERROR",
    "seed": 0xCC,
    "num_workers": 2,
    # Specify the environment to use, where "compiler_gym" is the name we 
    # passed to tune.register_env().
    "env": "compiler_gym",
    # Reduce the size of the batch/trajectory lengths to match our short 
    # training run.
    "rollout_fragment_length": 5,
    "train_batch_size": 5,
    "sgd_minibatch_size": 5,
    "gamma": 0.8, #tune.grid_search([0.5, 0.8, 0.9]), # def 0.99
    "lr": 1e-4, #tune.grid_search([0.01, 0.001, 0.0001]), # def 1e-4
    "horizon": 3, # def None
    "soft_horizon": True,
    "evaluation_interval": 5, # def None
    "evaluation_num_episodes": 1, # def 10
    "model": {'fcnet_hiddens': [5, 5]}
    # "model": {                            # The NN model we'll optimize.
    #     'fcnet_hiddens': [                # "Fully-connected network with N hidden layers".
    #         tune.grid_search([20, 40]),   # Try these four values for layer one.
    #         tune.grid_search([20, 40])    # Try these four values for layer two.
    #     ]
    # },
}

In [10]:
analysis = tune.run(
    PPOTrainer,
    fail_fast=True,
    checkpoint_at_end=True,
    # stop=TimeStopper(),
    stop={
        "episodes_total": 1000,
        # "episode_reward_mean": 30
    },
    config=PPO_CONFIG
)

wandb: WARNING Found log directory outside of given root_logdir, dropping given root_logdir for event file in /home/dejang/ray_results/PPOTrainer_2022-07-05_15-25-04/PPOTrainer_compiler_gym_2c379_00000_0_2022-07-05_15-25-05
(PPOTrainer pid=560340) 2022-07-05 15:25:08,982	INFO trainer.py:2332 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
(PPOTrainer pid=560340) 2022-07-05 15:25:09,198	WARNING deprecation.py:46 -- DeprecationWarning: `evaluation_num_episodes` has been deprecated. Use ``evaluation_duration` and `evaluation_duration_unit=episodes`` instead. This will raise an error in the future!
(PPOTrainer pid=560340) 2022-07-05 15:25:09,199	WARNING ppo.py:386 -- `train_batch_size` (5) cannot be achieved with your other settings (num_workers=2 num_envs_per_worker=1 rollout_fragment_len

Trial name,status,loc
PPOTrainer_compiler_gym_2c379_00000,RUNNING,100.37.253.28:560340


(PPOTrainer pid=560340) 2022-07-05 15:25:19,283	INFO trainable.py:159 -- Trainable.setup took 10.302 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(PPOTrainer pid=560340) 2022-07-05 15:25:19,284	WARNING util.py:65 -- Install gputil for GPU system monitoring.
(RolloutWorker pid=560394) E0705 15:25:19.297726 140252437153344 example_service.py:263] CRITICAL - 
(RolloutWorker pid=560394) 
(RolloutWorker pid=560394) Working_dir = /dev/shm/compiler_gym_dejang/s/0705T152513-338012-0236
(RolloutWorker pid=560394) 
(RolloutWorker pid=560393) E0705 15:25:19.297726 140432008803904 example_service.py:263] CRITICAL - 
(RolloutWorker pid=560393) 
(RolloutWorker pid=560393) Working_dir = /dev/shm/compiler_gym_dejang/s/0705T152513-337364-6f35
(RolloutWorker pid=560393) 


Result for PPOTrainer_compiler_gym_2c379_00000:
  agent_timesteps_total: 8
  counters:
    num_agent_steps_sampled: 8
    num_agent_steps_trained: 8
    num_env_steps_sampled: 8
    num_env_steps_trained: 8
  custom_metrics: {}
  date: 2022-07-05_15-25-23
  done: false
  episode_len_mean: 3.0
  episode_media: {}
  episode_reward_max: 0.016922388863661375
  episode_reward_mean: 0.01018992093293436
  episode_reward_min: 0.0034574530022073446
  episodes_this_iter: 2
  episodes_total: 2
  experiment_id: 1ca6e38840c340479addb8dd7cbbc3be
  hostname: codah
  info:
    learner:
      default_policy:
        custom_metrics: {}
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 9.999999747378752e-05
          entropy: 1.3862930536270142
          entropy_coeff: 0.0
          kl: 1.5437602769452496e-06
          model: {}
          policy_loss: -0.49426084756851196
          total_loss: -0.49416399002075195
          vf_explained_var: 7.748603536583687e-08
      

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_compiler_gym_2c379_00000,RUNNING,100.37.253.28:560340,1,4.42793,8,0.0101899,0.0169224,0.00345745,3


...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_compiler_gym_2c379_00000,RUNNING,100.37.253.28:560340,374,1469.98,2992,-0.329435,39.585,-38.4939,3


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_compiler_gym_2c379_00000,RUNNING,100.37.253.28:560340,374,1469.98,2992,-0.329435,39.585,-38.4939,3


Result for PPOTrainer_compiler_gym_2c379_00000:
  agent_timesteps_total: 3000
  counters:
    num_agent_steps_sampled: 3000
    num_agent_steps_trained: 3000
    num_env_steps_sampled: 3000
    num_env_steps_trained: 3000
  custom_metrics: {}
  date: 2022-07-05_15-50-12
  done: true
  episode_len_mean: 3.0
  episode_media: {}
  episode_reward_max: 39.58496904304526
  episode_reward_mean: 0.0710466199061537
  episode_reward_min: -38.49390607994264
  episodes_this_iter: 4
  episodes_total: 1000
  evaluation:
    custom_metrics: {}
    episode_len_mean: 3.0
    episode_media: {}
    episode_reward_max: 0.02068836848704647
    episode_reward_mean: 0.02068836848704647
    episode_reward_min: 0.02068836848704647
    episodes_this_iter: 1
    hist_stats:
      episode_lengths:
      - 3
      episode_reward:
      - 0.02068836848704647
    off_policy_estimator: {}
    policy_reward_max: {}
    policy_reward_mean: {}
    policy_reward_min: {}
    sampler_perf:
      mean_action_processing_ms: 

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_compiler_gym_2c379_00000,TERMINATED,100.37.253.28:560340,375,1476.08,3000,0.0710466,39.585,-38.4939,3


2022-07-05 15:50:12,878	INFO tune.py:747 -- Total run time: 1508.04 seconds (1507.31 seconds for the tuning loop).


In [11]:
checkpoint = analysis.get_best_checkpoint(
    metric="episode_reward_mean",
    mode="max",
    trial=analysis.trials[0]s
)

In [12]:
analysis.dataframe()

,episode_reward_max,episode_reward_min,episode_reward_mean,episode_len_mean,episodes_this_iter,num_healthy_workers,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_trained,...,config/log_level,config/lr,config/model,config/num_workers,config/rollout_fragment_length,config/seed,config/sgd_minibatch_size,config/soft_horizon,config/train_batch_size,logdir
0,39.584969,-38.493906,0.071047,3.0,4,2,3000,3000,3000,3000,...,ERROR,0.0001,"{'fcnet_hiddens': [5, 5]}",2,5,204,5,True,5,/home/dejang/ray_results/PPOTrainer_2022-07-05...


In [13]:
trial = analysis.get_best_trial(metric="episode_reward_mean", mode="max")
log_dir = analysis.get_best_logdir(metric="episode_reward_mean", mode="max")
print(log_dir)

/home/dejang/ray_results/PPOTrainer_2022-07-05_15-25-04/PPOTrainer_compiler_gym_2c379_00000_0_2022-07-05_15-25-05


In [14]:
trial.stopping_criterion

{'episodes_total': 1000}

In [15]:
trial.metric_analysis

{'episode_reward_max': {'max': 39.58496904304526,
  'min': 0.016922388863661375,
  'avg': 37.35342323484236,
  'last': 39.58496904304526,
  'last-5-avg': 39.58496904304526,
  'last-10-avg': 39.584969043045255},
 'episode_reward_min': {'max': 0.0034574530022073446,
  'min': -39.82938783525944,
  'avg': -37.717964688154716,
  'last': -38.49390607994264,
  'last-5-avg': -38.49390607994264,
  'last-10-avg': -38.49390607994264},
 'episode_reward_mean': {'max': 7.186053079470235,
  'min': -0.6798548492744083,
  'avg': 0.08435085538460046,
  'last': 0.0710466199061537,
  'last-5-avg': 0.014979303357484,
  'last-10-avg': -0.07999296632692823},
 'episode_len_mean': {'max': 3.0,
  'min': 3.0,
  'avg': 3.0000000000000004,
  'last': 3.0,
  'last-5-avg': 3.0,
  'last-10-avg': 3.0},
 'episodes_this_iter': {'max': 4,
  'min': 2,
  'avg': 2.6666666666666665,
  'last': 4,
  'last-5-avg': 2.8,
  'last-10-avg': 2.8},
 'num_healthy_workers': {'max': 2,
  'min': 2,
  'avg': 1.9999999999999962,
  'last': 2,

In [16]:
trial.checkpoint.value

'/home/dejang/ray_results/PPOTrainer_2022-07-05_15-25-04/PPOTrainer_compiler_gym_2c379_00000_0_2022-07-05_15-25-05/checkpoint_000375/checkpoint-375'

In [17]:
import ray.rllib.agents.ppo as ppo
ppo.DEFAULT_CONFIG

2022-07-05 15:50:13,696	WARNING deprecation.py:46 -- DeprecationWarning: `ray.rllib.agents.ppo.ppo.DEFAULT_CONFIG` has been deprecated. Use `ray.rllib.agents.ppo.ppo.PPOConfig(...)` instead. This will raise an error in the future!


{'num_workers': 2,
 'num_envs_per_worker': 1,
 'create_env_on_driver': False,
 'rollout_fragment_length': 200,
 'batch_mode': 'truncate_episodes',
 'gamma': 0.99,
 'lr': 5e-05,
 'train_batch_size': 4000,
 'model': {'_use_default_native_models': False,
  '_disable_preprocessor_api': False,
  '_disable_action_flattening': False,
  'fcnet_hiddens': [256, 256],
  'fcnet_activation': 'tanh',
  'conv_filters': None,
  'conv_activation': 'relu',
  'post_fcnet_hiddens': [],
  'post_fcnet_activation': 'relu',
  'free_log_std': False,
  'no_final_linear': False,
  'vf_share_layers': False,
  'use_lstm': False,
  'max_seq_len': 20,
  'lstm_cell_size': 256,
  'lstm_use_prev_action': False,
  'lstm_use_prev_reward': False,
  '_time_major': False,
  'use_attention': False,
  'attention_num_transformer_units': 1,
  'attention_dim': 64,
  'attention_num_heads': 1,
  'attention_head_dim': 32,
  'attention_memory_inference': 50,
  'attention_memory_training': 50,
  'attention_position_wise_mlp_dim': 32,

In [18]:
trial.checkpoint.value

'/home/dejang/ray_results/PPOTrainer_2022-07-05_15-25-04/PPOTrainer_compiler_gym_2c379_00000_0_2022-07-05_15-25-05/checkpoint_000375/checkpoint-375'

In [19]:
trial.config

{'log_level': 'ERROR',
 'seed': 204,
 'num_workers': 2,
 'env': 'compiler_gym',
 'rollout_fragment_length': 5,
 'train_batch_size': 5,
 'sgd_minibatch_size': 5,
 'gamma': 0.8,
 'lr': 0.0001,
 'horizon': 3,
 'soft_horizon': True,
 'evaluation_interval': 5,
 'evaluation_num_episodes': 1,
 'model': {'fcnet_hiddens': [5, 5]}}

In [20]:
trainer = PPOTrainer(config=trial.config, env="compiler_gym")
trainer.restore(trial.checkpoint.value)

wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`
wandb: WARNING Found log directory outside of given root_logdir, dropping given root_logdir for event file in /home/dejang/ray_results/PPOTrainer_compiler_gym_2022-07-05_15-50-14w1lcst39
2022-07-05 15:50:14,025	INFO trainer.py:2332 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2022-07-05 15:50:14,027	WARNING deprecation.py:46 -- DeprecationWarning: `evaluation_num_episodes` has been deprecated. Use ``evaluation_duration` and `evaluation_duration_unit=episodes`` instead. This will raise an error in the future!
2022-07-05 15:50:14,028	WARNING ppo.py:386 -- `train_batch_size` (5) cannot be achieved with your other settings (num_workers=2 num_envs_per_work

In [21]:
model = trainer.get_policy().model
policy = trainer.get_policy()

In [22]:
model.model_config

{'_use_default_native_models': False,
 '_disable_preprocessor_api': False,
 '_disable_action_flattening': False,
 'fcnet_hiddens': [5, 5],
 'fcnet_activation': 'tanh',
 'conv_filters': None,
 'conv_activation': 'relu',
 'post_fcnet_hiddens': [],
 'post_fcnet_activation': 'relu',
 'free_log_std': False,
 'no_final_linear': False,
 'vf_share_layers': False,
 'use_lstm': False,
 'max_seq_len': 20,
 'lstm_cell_size': 256,
 'lstm_use_prev_action': False,
 'lstm_use_prev_reward': False,
 '_time_major': False,
 'use_attention': False,
 'attention_num_transformer_units': 1,
 'attention_dim': 64,
 'attention_num_heads': 1,
 'attention_head_dim': 32,
 'attention_memory_inference': 50,
 'attention_memory_training': 50,
 'attention_position_wise_mlp_dim': 32,
 'attention_init_gru_gate_bias': 2.0,
 'attention_use_n_prev_actions': 0,
 'attention_use_n_prev_rewards': 0,
 'framestack': True,
 'dim': 84,
 'grayscale': False,
 'zero_mean': True,
 'custom_model': None,
 'custom_model_config': {},
 'custo

In [23]:
def run_rollout (agent, env, n_iter=1, max_steps=5, verbose=False):
    """
    iterate through `n_iter` episodes in a rollout to emulate deployment in a production use case
    """
    for episode in range(n_iter):
        state = env.reset()
        sum_reward = 0

        for step in range(max_steps):
            try:
                
                action = int(agent.compute_single_action(state, explore=False))
                print(f"Compute action = {env.action_space.to_string(action)}")

                state, reward, done, info = env.step(action)
                sum_reward += reward
                print(f"Compute reward = {reward}")

                if verbose:
                    print("reward {:6.3f}  sum {:6.3f}".format(reward, sum_reward))
                    env.render()
            except Exception as e:
                print(f'----------------------------> Exception = {e}')
                break


        # report at the end of each episode
        print("CUMULATIVE REWARD:", round(sum_reward, 3), "\n")
        yield sum_reward


In [24]:
# trainer.restore(trial.checkpoint.value)
history = []
for episode_reward in run_rollout(trainer, env, verbose=False):
    history.append(episode_reward)
    
print("average reward:", round(sum(history) / len(history), 3))

E0705 15:50:25.025571 140184553244224 example_service.py:263] CRITICAL - 

Working_dir = /dev/shm/compiler_gym_dejang/s/0705T152501-249505-dbfd



Compute action = swap_down
Compute reward = -0.10795806245708439
Compute action = swap_down
Compute reward = -1.2608128186439285
Compute action = swap_down
Compute reward = -0.06702328541776437
Compute action = swap_down
Compute reward = -0.0016316470605532984
Compute action = swap_down
Compute reward = 0.006835741497085546
CUMULATIVE REWARD: -1.431 

average reward: -1.431


In [25]:
# If running in a notebook, finish the wandb run to upload the tensorboard logs to W&B
wandb.finish()
ray.shutdown()

global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
ray/tune/agent_timesteps_total,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
ray/tune/counters/num_agent_steps_sampled,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
ray/tune/counters/num_agent_steps_trained,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
ray/tune/counters/num_env_steps_sampled,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
ray/tune/counters/num_env_steps_trained,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
ray/tune/done,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
ray/tune/episode_len_mean,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
ray/tune/episode_reward_max,▁▄▄▄▅▅███▇████▇▇▇▇▄▄▄▅▅▅████▇▇▇▇▇▇▅▇▇▇██
ray/tune/episode_reward_mean,█▅▂▄▁▃▂▃▁▁▂▁▄▃▂▁▃▁▁▄▁▄▂▂▂▂▁▂▁▃▃▂▂▄▂▁▁▃▁▂
ray/tune/episode_reward_min,██▅▅▄▄▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▅▅▂▂▂▂▂▂▂▂▂▂▂▂▂▂
